In [ ]:
"""総合演習2 マクロ系シミュレーション 第2回 例題1

2次元非定常拡散方程式のシミュレーション(有限差分法)
"""

import sys
from typing import Final, Self

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML
from matplotlib import animation

# ========== パラメータ ==========
LX: Final[float] = 10              # 部屋のx方向の長さ(m)
LY: Final[float] = LX              # 部屋のy方向の長さ(m)
D: Final[float] = 10**(-2)         # 二酸化炭素の拡散係数(m^2/s)
C_OUT: Final[float] = 400            # 外気の二酸化炭素濃度(ppm)

NX: Final[int] = 31                # x方向の格子点数
NY: Final[int] = NX                # y方向の格子点数
DT: Final[float] = 3               # 時間刻み幅(s)
T_END: Final[float] = 1000         # シミュレーション時間(s)
DT_PLOT: Final[float] = T_END/50   # 等値線図を作成する時間間隔(s)
# ==============================

# float型のNumPy配列の型エイリアス
FloatArray = np.typing.NDArray[np.float64]


def func_src(x: float, y: float, lx: float, ly: float) -> float:
    """(x,y)における二酸化炭素の生成項を計算する

    Parameters
    ----------
    x : float
        x座標
    y : float
        y座標
    lx : float
        部屋のx方向の長さ
    ly : float
        部屋のy方向の長さ

    Returns
    -------
    float
        (x,y)における二酸化炭素の生成項
    """

    return 1


class Variable:
    """格子点上の物理量を扱うクラス

    Attributes
    ----------
    value : FloatArray
        格子点上の物理量の値
    __name : str
        'grid_x', 'grid_y', 'co2', 'src' のいずれか

    Notes
    -----
    インスタンスを生成する前に, set_class_variablesクラスメソッドを実行する必要あり
    Variableクラスの外で使用しない属性は, 変数名の最初にダンダー(__)をつけて隠蔽している
    """

    __lx: float
    __ly: float
    __nx: int
    __ny: int
    __dx: float
    __dy: float

    # set_class_variablesクラスメソッドを実行したかどうかを判定する真偽値
    __flag: bool = False

    @classmethod
    def set_class_variables(cls, lx: float, ly: float, nx: int, ny: int) -> None:
        """Variableクラスのクラス変数を設定する

        Parameters
        ----------
        lx : float
            部屋のx方向の長さ
        ly : float
            部屋のy方向の長さ
        nx : int
            x方向の格子点数
        ny : int
            y方向の格子点数
        """

        cls.__lx = lx
        cls.__ly = ly
        cls.__nx = nx
        cls.__ny = ny
        cls.__dx = lx / (nx-1)
        cls.__dy = ly / (ny-1)
        cls.__flag = True

    @classmethod
    def check_stable(cls, dt: float, d: float):
        """数値的安定性をチェックする
        
        数値的安定性の条件:
            時間刻み幅 < 拡散によって情報が格子点間を伝わるのにかかる時間

        Parameters
        ----------
        dt : float
            時間刻み幅
        d : float
            二酸化炭素の拡散係数
        """

        if not Variable.__flag:
            print('[ERROR] set_class_variablesクラスメソッドが実行されていません')
            sys.exit()

        dx: float = Variable.__dx
        dy: float = Variable.__dy
        dt_stable: float = 1/((2*d)*(1/(dx**2)+1/(dy**2)))
        if dt > dt_stable:
            print(f'[ERROR] 数値的安定性の条件(DT < {dt_stable})を満たしていません')
            sys.exit()

    def __init__(self, name: str) -> None:
        """Variableクラスのイニシャライザ

        Parameters
        ----------
        name : str
            'grid_x', 'grid_y', 'co2', 'src' のいずれか
        """

        if not Variable.__flag:
            print('[ERROR] set_class_variablesクラスメソッドが実行されていません')
            sys.exit()

        if name not in ['grid_x', 'grid_y', 'co2', 'src']:
            print('[ERROR] Variableクラスのイニシャライザの引数が不適切です')
            sys.exit()
        self.__name: str = name

        nx: int = Variable.__nx
        ny: int = Variable.__ny
        self.value: FloatArray = np.empty((nx, ny), dtype=np.float64)

        lx: float = Variable.__lx
        ly: float = Variable.__ly
        dx: float = Variable.__dx
        dy: float = Variable.__dy
        x: float
        y: float
        for ix in range(nx):
            x = dx * ix
            for iy in range(ny):
                y = dy * iy
                self.value[ix, iy] = self.__set_initial_condition(x, y, lx, ly)

    def __set_initial_condition(self, x: float, y: float, lx: float, ly: float) -> float:
        """(x,y)における(初期条件の)値を設定する

        Parameters
        ----------
        x : float
            x座標
        y : float
            y座標
        lx : float
            部屋のx方向の長さ
        ly : float
            部屋のy方向の長さ

        Returns
        -------
        float
            (x,y)における(初期条件の)値
        """

        if self.__name == 'grid_x':
            return x
        elif self.__name == 'grid_y':
            return y
        elif self.__name == 'co2':
            return 5000
        elif self.__name == 'src':
            return func_src(x, y, lx, ly)

    def set_boundary_condition(self, src: Self, d: float, c_out: float, dt: float) -> None:
        """二酸化炭素濃度の境界条件を設定する

        Parameters
        ----------
        src : Self
            Variableクラスのインスタンス(src)
        d : float
            二酸化炭素の拡散係数
        c_out : float
            外気の二酸化炭素濃度
        dt : float
            時間刻み幅
        """

        if self.__name == 'co2':

            dx: float = Variable.__dx
            dy: float = Variable.__dy

            # 上端
            self.value[:, -1] = c_out

            # 左端
            self.value[0, 1:-1] += (
                d * (2 * (self.value[1, 1:-1] - self.value[0, 1:-1]) / (dx**2)
                        + (self.value[0, 2:] - 2 * self.value[0, 1:-1] + self.value[0, :-2]) / (dy**2))
                + src.value[0, 1:-1]
            ) * dt
                
            # 左下端
            self.value[0, 0] += (
                d * (2 * (self.value[1, 0] - self.value[0, 0]) / (dx**2)
                     + 2 * (self.value[0, 1] - self.value[0, 0]) / (dy**2))
                + src.value[0, 0]
            ) * dt

            # 下端
            self.value[1:-1, 0] += (
                d * ((self.value[2:, 0] - 2 * self.value[1:-1, 0] + self.value[:-2, 0]) / (dx**2)
                        + 2 * (self.value[1:-1, 1] - self.value[1:-1, 0]) / (dy**2))
                + src.value[1:-1, 0]
            ) * dt

            # 右下端
            self.value[-1, 0] += (
                d * (2 * (self.value[-2, 0] - self.value[-1, 0]) / (dx**2)
                     + 2 * (self.value[-1, 1] - self.value[-1, 0]) / (dy**2))
                + src.value[-1, 0]
            ) * dt

            # 右端
            self.value[-1, 1:-1] += (
                d * (2 * (self.value[-2, 1:-1] - self.value[-1, 1:-1]) / (dx**2)
                        + (self.value[-1, 2:] - 2 * self.value[-1, 1:-1] + self.value[-1, :-2]) / (dy**2))
                + src.value[-1, 1:-1]
            ) * dt

    def laplacian(self) -> FloatArray:
        """ラプラシアンを計算する

        Returns
        -------
        laplacian : FloatArray
            ラプラシアン
        """

        nx: int = Variable.__nx
        ny: int = Variable.__ny
        laplacian: FloatArray = np.zeros((nx, ny), dtype=np.float64)

        dx: float = Variable.__dx
        dy: float = Variable.__dy
        for ix in range(1, nx-1):
            for iy in range(1, ny-1):
                laplacian[ix, iy] = (
                    (self.value[ix+1, iy] - 2 * self.value[ix, iy] + self.value[ix-1, iy]) / (dx**2)
                    + (self.value[ix, iy+1] - 2 * self.value[ix, iy] + self.value[ix, iy-1]) / (dy**2)
                )
        return laplacian
    

def create_plot(frame: int, *fargs) -> tuple:
    """各タイムステップでの等値線図を作成する
    
    animation.FuncAnimationに使用する関数
    
    Parameters
    ----------
    frame : int
        フレーム番号
    fargs : tuple
        格子点のx座標, 格子点のy座標, 時刻, co2の結果, srcを格納したタプル
    """

    grid_x: FloatArray
    grid_y: FloatArray
    results_t: list[float]
    results_co2: list[FloatArray]
    results_src: list[FloatArray]
    grid_x, grid_y, results_t, results_co2, results_src = fargs
    
    axes[0].cla()
    axes[1].cla()

    # co2の等値線図
    co2: FloatArray = results_co2[frame]
    vmin: float = C_OUT
    vmax: float = np.max(results_co2)
    im1 = axes[0].contourf(grid_x, grid_y, co2, vmin=vmin, vmax=vmax, levels=20, cmap='jet')
    axes[0].contour(im1, colors='k', linewidths=0.5)
    axes[0].set_xlabel('x')
    axes[0].set_ylabel('y')
    axes[0].set_title('CO2')
    axes[0].set_aspect('equal')

    # srcの等値線図
    src: FloatArray = results_src[frame]
    vmin = 0
    vmax = np.max(results_src)
    im2 = axes[1].contourf(grid_x, grid_y, src, vmin=vmin, vmax=vmax, levels=20, cmap='Greens')
    axes[1].contour(im2, colors='k', linewidths=0.5)
    axes[1].set_xlabel('x')
    axes[1].set_ylabel('y')
    axes[1].set_title('S0')
    axes[1].set_aspect('equal') 

    figure.suptitle(f't = {results_t[frame]:.1f} sec')

    return im1, im2


if __name__ == '__main__':

    Variable.set_class_variables(LX, LY, NX, NY)
    # Variable.check_stable(DT, D)

    # Variableクラスのインスタンスとして変数を準備
    grid_x: Variable = Variable('grid_x')
    grid_y: Variable = Variable('grid_y')
    co2: Variable = Variable('co2')
    src: Variable = Variable('src')

    t: float = 0
    it: int = 0

    # アニメーションの準備
    figure, axes = plt.subplots(1, 2, figsize=(10, 5))
    results_t: list[float] = [t]
    results_co2: list[FloatArray] = [co2.value.copy()]
    results_src: list[FloatArray] = [src.value.copy()]

    while t < T_END:

        # 方程式にしたがって時間発展(Euler法)
        co2.value += (D * co2.laplacian() + src.value) * DT
        co2.set_boundary_condition(src, D, C_OUT, DT)
        t += DT

        # アニメーション用に結果を保存
        if it % int(DT_PLOT / DT) == 0:
            results_t.append(t)
            results_co2.append(co2.value.copy())
            results_src.append(src.value.copy())
        it += 1

    # アニメーションを作成
    im1, im2 = create_plot(len(results_t)-1, grid_x.value, grid_y.value, results_t, results_co2, results_src)
    figure.colorbar(im1, ax=axes[0])
    figure.colorbar(im2, ax=axes[1])
    figure.tight_layout()
    anim = animation.FuncAnimation(figure, create_plot, range(len(results_t)),
                                   fargs=(grid_x.value, grid_y.value, results_t, results_co2, results_src))
    display(HTML(anim.to_jshtml()))
    plt.close()